# COVID-19 Decision Stockpile Analysis
To use the surge model, change the slider for stockpile days and the table below will change to indicate minimum inventory required.

Use caution when interpreting these numbers

In [7]:
import pandas as pd
import ipywidgets as widgets
import sys

sys.path.append('../src')
from restart import NoteCompose
from output import Output
from ipywidgets import Layout

from util import set_config

import ipysheet
import numpy

def display_sheet(sheet):
    df = to_df(sheet)
    index_name = df.index.name
    headers = list(df.index)
    df.insert(loc=0, column=index_name, value=headers)
    sheet = ipysheet.pandas_loader.from_dataframe(df)
    sheet.layout = Layout(max_height='300px', overflow_y='scroll')
    sheet.row_headers = False
    return sheet

def to_sheet(df):
    return ipysheet.pandas_loader.from_dataframe(df)

def to_df(sheet):
    return ipysheet.pandas_loader.to_dataframe(sheet)

def read_only(sheet):
    for cell in sheet.cells:
        setattr(cell, 'read_only', True)

def to_config(sheet):
    new_config['Data']['Demand m']['Level to Resource mn'] = np.array(to_df(sheet)).tolist()
    return new_config
    
def on_change(change): 
    new_sheet = sheet

## Bootstrap the Model
Select the parameters here to initialize the model (not implemented)

In [8]:
config = set_config('../src')
restart = NoteCompose(configdir='../src', population='oes', state='California', demand='washington')
population = restart.model.population
resource = restart.model.resource
demand = restart.model.demand

## Inventory Calculations

In [9]:
def set_stock(days):
    resource.demand(resource.inventory_ln_df)
    resource.set_inv_min(demand.level_total_demand_ln_df, days)
    stockpile = to_sheet(resource.inventory_ln_df)
    sheet = display_sheet(stockpile)
    display(sheet)
    
widgets.interact(set_stock, days=30, continuous_update=False)

interactive(children=(IntSlider(value=30, description='days', max=90, min=-30), Output()), _dom_classes=('widg…

<function __main__.set_stock(days)>

## Editable Burn Rates
These are burn rates per day for each resource, based on Washington protection level.

In [10]:
burn_df = demand.level_to_res_mn_df
burn = to_sheet(burn_df)

widgets.VBox([burn])

## Population Details
Right now the only population attribute is size, but there will be more.

In [11]:
pop_df = population.detail_pd_df
pop = to_sheet(pop_df)
read_only(pop)

display_pop = display_sheet(pop)
widgets.VBox([display_pop])

## Population Levels
Protection levels for the given population.

In [12]:
level_df = population.level_pm_df

levels = to_sheet(level_df)
levels.stretch_headers = 'none'
read_only(levels)

widgets.VBox([display_sheet(levels)])

# Population Demand per Resource
The daily burn rates of all resources for each subpopulation

In [13]:
demand_pn_df = demand.demand_pn_df
demand_sheet = to_sheet(demand_pn_df)
read_only(demand_sheet)

widgets.VBox([display_sheet(demand_sheet)])

## Experimenting with Vuetify

In [1]:
# import ipyvuetify as v

# vslider = v.Slider(value=30, thumb_label=True, min=0, max=120)
# vslider

# v.Container(children = [
#     display_sheet(demand_sheet)
# ])